In [5]:
from pyspark.sql import SparkSession

In [6]:
import pandas as pd

In [7]:
import numpy as np

In [8]:
from pyspark.sql.functions import sum,min,count

In [9]:
#import os
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/ubuntu/notebooks/aws-java-sdk-1.11.485.jar,/home/ubuntu/notebooks/hadoop-common-3.1.0.jar,/home/ubuntu/notebooks/hadoop-aws-3.1.0.jar pyspark-shell'

#import os
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk:1.10.34,org.apache.hadoop:hadoop-aws:2.6.0,com.databricks:spark-csv_2.11:1.4.0 pyspark-shell'

In [10]:
spark = SparkSession.builder.getOrCreate()

In [11]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

In [12]:
# Wbudowane funkcje, które mogą być przydatne:
# - udf: służy do tworzenia funkcji użytkownika
# - trim: usuwa białe spacje z początku i końca stringu
# - isnull: testuje, czy wartość jest pusta
from pyspark.sql.functions import col, udf, trim, isnull
from pyspark.sql.types import FloatType, IntegerType

In [13]:
def to_float (s) :
   return float(s.replace(',','.'))
float_udf = udf(to_float , FloatType())

In [14]:
#smog2_df=spark.read.json("s3a://openaq-fetches/realtime/2019-02-01/1548979266.ndjson")
smog2_df=spark.read.json("data/1580631065.ndjson")
smog2_df.limit(10).toPandas()

,attribution,averagingPeriod,city,coordinates,country,date,location,mobile,parameter,sourceName,sourceType,unit,value
0,"[(Health Protection Service, ACT Government, h...","(hours, 1.0)",Canberra,"(-35.220606, 149.043539)",AU,"(2020-02-02T18:00:00+11:00, 2020-02-02T07:00:0...",Florey,False,no2,Australia - ACT,government,ppm,0.000
1,"[(Health Protection Service, ACT Government, h...","(hours, 1.0)",Canberra,"(-35.220606, 149.043539)",AU,"(2020-02-02T18:00:00+11:00, 2020-02-02T07:00:0...",Florey,False,o3,Australia - ACT,government,ppm,0.035
2,"[(Health Protection Service, ACT Government, h...","(hours, 1.0)",Canberra,"(-35.220606, 149.043539)",AU,"(2020-02-02T18:00:00+11:00, 2020-02-02T07:00:0...",Florey,False,co,Australia - ACT,government,ppm,0.190
3,"[(Health Protection Service, ACT Government, h...","(hours, 1.0)",Canberra,"(-35.220606, 149.043539)",AU,"(2020-02-02T18:00:00+11:00, 2020-02-02T07:00:0...",Florey,False,pm10,Australia - ACT,government,µg/m³,91.540
4,"[(Health Protection Service, ACT Government, h...","(hours, 1.0)",Canberra,"(-35.220606, 149.043539)",AU,"(2020-02-02T18:00:00+11:00, 2020-02-02T07:00:0...",Florey,False,pm25,Australia - ACT,government,µg/m³,23.730
5,"[(Health Protection Service, ACT Government, h...","(hours, 1.0)",Canberra,"(-35.418302, 149.094018)",AU,"(2020-02-02T18:00:00+11:00, 2020-02-02T07:00:0...",Monash,False,no2,Australia - ACT,government,ppm,0.000
6,"[(Health Protection Service, ACT Government, h...","(hours, 1.0)",Canberra,"(-35.418302, 149.094018)",AU,"(2020-02-02T18:00:00+11:00, 2020-02-02T07:00:0...",Monash,False,o3,Australia - ACT,government,ppm,0.034
7,"[(Health Protection Service, ACT Government, h...","(hours, 1.0)",Canberra,"(-35.418302, 149.094018)",AU,"(2020-02-02T18:00:00+11:00, 2020-02-02T07:00:0...",Monash,False,co,Australia - ACT,government,ppm,0.040
8,"[(Health Protection Service, ACT Government, h...","(hours, 1.0)",Canberra,"(-35.418302, 149.094018)",AU,"(2020-02-02T18:00:00+11:00, 2020-02-02T07:00:0...",Monash,False,pm10,Australia - ACT,government,µg/m³,79.380
9,"[(Health Protection Service, ACT Government, h...","(hours, 1.0)",Canberra,"(-35.418302, 149.094018)",AU,"(2020-02-02T18:00:00+11:00, 2020-02-02T07:00:0...",Monash,False,pm25,Australia - ACT,government,µg/m³,22.250


In [15]:
smog2_df.printSchema()

root
 |-- attribution: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |-- averagingPeriod: struct (nullable = true)
 |    |-- unit: string (nullable = true)
 |    |-- value: double (nullable = true)
 |-- city: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- date: struct (nullable = true)
 |    |-- local: string (nullable = true)
 |    |-- utc: string (nullable = true)
 |-- location: string (nullable = true)
 |-- mobile: boolean (nullable = true)
 |-- parameter: string (nullable = true)
 |-- sourceName: string (nullable = true)
 |-- sourceType: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- value: double (nullable = true)



In [17]:
#“Spłaszczenie” schematu danych ładowanych z JSON:
smog3_df=smog2_df.select("location", "city", "parameter", "unit", "value",\
                         col("date.local").alias('date_local'),\
                         col('date.utc').alias('date_utc'),\
                         'coordinates.*')
smog3_df.printSchema()

root
 |-- location: string (nullable = true)
 |-- city: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- value: double (nullable = true)
 |-- date_local: string (nullable = true)
 |-- date_utc: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [18]:
%%time
# OPIS formatu danych: https://github.com/openaq/openaq-data-format
#https://openaq-fetches.s3.amazonaws.com/index.html 
#smog_df = spark.read.csv("s3a://openaq-fetches/daily/2017-08-1*.csv", inferSchema=True, header=False).\
#            toDF('city', 'value', 'unit', 'parameter', 'country', 'region1', 'region2', 'date1', 'date2',\
#                  'sourceType', 'mobile', 'latitude', 'longitude', 'avgPeriod', 'avgPeriodUnits')

smog_df = spark.read.csv("data/2017-08-1*.csv", inferSchema=True, header=False).\
    toDF('city', 'value', 'unit', 'parameter', 'country', 'region1', 'region2', 'date1', 'date2',\
         'sourceType', 'mobile', 'latitude', 'longitude', 'avgPeriod', 'avgPeriodUnits')

CPU times: user 10 ms, sys: 10 ms, total: 20 ms
Wall time: 46.9 s


In [19]:
smog_df.printSchema()
print(smog_df.count())
smog_df.limit(100).toPandas()

root
 |-- city: string (nullable = true)
 |-- value: double (nullable = true)
 |-- unit: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- country: string (nullable = true)
 |-- region1: string (nullable = true)
 |-- region2: string (nullable = true)
 |-- date1: timestamp (nullable = true)
 |-- date2: timestamp (nullable = true)
 |-- sourceType: string (nullable = true)
 |-- mobile: boolean (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- avgPeriod: double (nullable = true)
 |-- avgPeriodUnits: string (nullable = true)

4533361


,city,value,unit,parameter,country,region1,region2,date1,date2,sourceType,mobile,latitude,longitude,avgPeriod,avgPeriodUnits
0,Smithton,4.00,µg/m³,pm25,AU,Tasmania Region,Australia - Tasmania,2017-08-14 23:44:23,2017-08-14 23:44:23,government,False,-40.850500,145.133000,0.25,hours
1,Smithton,20.00,µg/m³,pm10,AU,Tasmania Region,Australia - Tasmania,2017-08-14 23:44:23,2017-08-14 23:44:23,government,False,-40.850500,145.133000,0.25,hours
2,Wynyard,5.00,µg/m³,pm25,AU,Tasmania Region,Australia - Tasmania,2017-08-14 23:44:24,2017-08-14 23:44:24,government,False,-40.991800,145.719000,0.25,hours
3,Wynyard,19.00,µg/m³,pm10,AU,Tasmania Region,Australia - Tasmania,2017-08-14 23:44:24,2017-08-14 23:44:24,government,False,-40.991800,145.719000,0.25,hours
4,Emu River,6.00,µg/m³,pm25,AU,Tasmania Region,Australia - Tasmania,2017-08-14 23:44:20,2017-08-14 23:44:20,government,False,-41.108100,145.883000,0.25,hours
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Centro Quintero,11.19,µg/m³,no2,CL,Quintero,Chile - SINCA,2017-08-14 22:00:00,2017-08-14 22:00:00,government,False,-32.786170,-71.531435,NaN,None
96,Centro Quintero,14.26,µg/m³,so2,CL,Quintero,Chile - SINCA,2017-08-14 23:00:00,2017-08-14 23:00:00,government,False,-32.786170,-71.531435,NaN,None
97,Quintero,8.64,µg/m³,so2,CL,Quintero,Chile - SINCA,2017-08-14 23:00:00,2017-08-14 23:00:00,government,False,-32.772516,-71.535303,NaN,None
98,Ventanas,13.32,µg/m³,pm25,CL,Puchuncaví,Chile - SINCA,2017-08-14 23:00:00,2017-08-14 23:00:00,government,False,-32.778988,-71.373031,NaN,None


In [20]:
# show all the different measured parameters
smog_df.select('parameter').distinct().toPandas()

,parameter
0,bc
1,so2
2,co
3,o3
4,pm10
5,no2
6,pm25


# Zadanie 1. Znalezc miasto, w ktorym najnizsze zanieczyszczenie w roku jest najwieksze sposrod wszystkich miast

In [30]:
from pyspark.sql.functions import max
smog_df.select('value').toPandas()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44473)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", 

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44473)

In [22]:
smog_df.filter(smog_df.value == smog_df.value.max())

TypeError: 'Column' object is not callable

# Zadanie 2: Ranking miast pod wzgledem liczby dni w roku, w ktorych poziom jest wyzszy niz X

In [32]:
# Etap 1: dodać kolumnę z datą dniową na podstawie 'date1'
# przydadzą się następujące funkcje:
from pyspark.sql.functions import date_format, to_date, countDistinct

In [33]:
# Etap 2: stworzyć ramkę zawężoną do pomiarów pm25 o wartości większej niż np. 35.0


In [34]:
# Etap 3: stworzyć ranking: pogrupować po miastach, dla każdej grupy obliczając liczbę dni

In [14]:
# Sprawdzenie: wybrać z ramki z etapu 2 jedno miasto, np. Delhi, obliczyć liczbę dni i porównać wynik